## Import Libraries

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from Utils import homepath, compute_returns

import warnings
warnings.filterwarnings("ignore")

## Data

In [2]:
sp500_prices = pd.read_csv(homepath + "data/sp500_timeseries.txt", sep='\t', parse_dates=['Date'], index_col='Date')
sp500_index = pd.read_csv(homepath + "data/indices.csv", parse_dates=["Date"], index_col="Date")["sp500"]

In [4]:
rets = compute_returns(sp500_prices)

## Historical Analysis

In [ ]:
means = rets["weekly_log"].mean()
vols = rets["weekly_log"].std()

A       0.038397
ABBV    0.036539
ABT     0.035699
ACN     0.039144
ADBE    0.051689
          ...   
XYL     0.043138
YUM     0.039249
ZBH     0.042725
ZBRA    0.054012
ZTS     0.040080
Length: 462, dtype: float64